In [1]:
import vectorbt as vbt
import os
import sys
import numpy as np
import pandas as pd
from numba import njit
from plotly.subplots import make_subplots

from strategies.SellOff.SellOff import create_portfolio, plots_from_portfolio

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lib.utils import where_true_set_series

In [ ]:
#vbt.settings.caching['enabled'] = False
symbol = "ADA"
candles = "_25000"
file = f"/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/{candles[1:]}/Binance_{symbol}USDT_minute{candles}.csv"

lr_thld = -np.linspace(0, 3, 30, endpoint=True)
vol_thld = np.linspace(0, 4, 30, endpoint=True)
lag = list(range(6, 100, 2))

fee = 0.001

port = create_portfolio(file, fee, lr_thld, vol_thld, lag)

In [ ]:
min_trades = 500
min_lr = 0.0
parameters_to_save = {
    "lr_thld": f"[{lr_thld[0]},{lr_thld[-1]}]",
    "vol_thld": f"[{vol_thld[0]},{vol_thld[-1]}]",
    "lag": f"[{lag[0]},{lag[-1]}]",
    "fee": fee,
    "min_trades":min_trades,
    "min_lr = 0.0": min_lr
}
save_dir = f"{symbol}{candles}"
plots_from_portfolio(port, min_trades=min_trades, min_lr=min_lr, save_dir=save_dir, parameters_to_save=parameters_to_save)


In [ ]:
best_indexes = entry_exit_prob_avg.nlargest(20, keep="all")

# graficamos con el mejor de los resultados de la optimización
best_idx = best_indexes[-1]
best_lr_thld, best_vol_thld, best_lag = best_idx

In [ ]:
# recalculamos algunas cosas para graficar y entender lo que está pasando
ups, downs, lr_entries, lr_exits, vol_thld_std, vol_entries = signal_calculations(
    lr_ind.lr.to_numpy(),
    ma_mstd.lr_ma[best_lag].to_numpy(),
    ma_mstd.lr_mstd[best_lag].to_numpy(),
    volume.to_numpy(),
    ma_mstd.vol_mstd[best_lag].to_numpy(),
    best_lr_thld,
    best_vol_thld)

In [ ]:
def plot_series_vs_scatters(series_list: list, booleans_list):
    index = None
    series = series_list.pop(0)
    fig = series.vbt.plot()
    while len(series_list):
        series = series_list.pop(0)
        if not isinstance(series, pd.Series):
            series = pd.Series(series, index=index, copy=True)
        elif index is None:
            index = series.index
        series.vbt.plot(fig=fig)
    i = 1
    for scatter in booleans_list:
        if not isinstance(scatter, pd.Series):
            scatter = pd.Series(scatter, index=index, copy=True)
        elif index is None:
            index = series.index
        scatter = where_true_set_series(series, scatter)
        scatter.name = i
        i += 1
        fig = scatter.vbt.scatterplot(fig=fig)
    return fig

entries = signals.entries[best_idx]
plot_series_vs_scatters([lr_ind.lr, downs], []).show()

In [ ]:
plots = [
    plot_series_vs_scatters([lr_ind.lr, downs.T], []) #, downs, ups], [lr_entries, lr_exits]),
    #plot_series_vs_scatters([volume, vol_thld_std], [vol_entries]),
    #(entries.where(entries == True, np.nan)).vbt.scatterplot()
]

In [ ]:
def add_all_subplots(fig, row, col, list):
    for a in list:
        fig.add_trace(a, row=row, col=col)

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.02)
for i in range(len(plots)):
    add_all_subplots(fig, i+1, 1, plots[i].data)

fig.update_layout(height=700, legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [ ]:
@njit
def double_multiplier_nb(values, x, y):
    return values*x, values*y

DOUBLE_MULTIPLIER = vbt.IndicatorFactory(
    input_names=['values'],
    param_names=['x', 'y'],
    output_names=['x_mu','y_mu']
).from_apply_func(double_multiplier_nb)
x = np.linspace(0,2,5, endpoint=True)
y = -np.linspace(0,2,5, endpoint=True)
tp_sl = DOUBLE_MULTIPLIER.run(mstd_ind.mstd, x, y, param_product=True, short_name="tp_sl")

In [ ]:
tp_exits = lr_ind.lr_above(tp_sl.x_mu)
sl_exits = lr_ind.lr_below(tp_sl.y_mu)
final_exits = tp_exits.vbt | sl_exits.vbt
final_exits.columns = final_exits.columns.rename("lag", level=-1)
final_exits.head()
